# Learning of Process Representations Using Recurrent Neural Networks

In [11]:
import numpy as np
import pandas as pd

from replearn.eventlog import EventLog

from replearn.embedding_predict import EmbeddingPredict
from replearn.autoencoder import AutoencoderRepresentation
from replearn.doc2vec import Doc2VecRepresentation

from replearn.clustering import Clustering
from replearn.evaluation import Evaluation

## Load event log

In [14]:
# event log configuration
event_log_path = '../logs/multi-perspective_02'
file_name = 'p2p_500_10_20_5_2_1-0.3-1.json.gz'

case_attributes = None # auto-detect attributes
event_attributes = ['concept:name', 'user'] # use activity name and user
true_cluster_label = 'cluster'

# load file
event_log = EventLog(file_name, case_attributes=case_attributes, event_attributes=event_attributes, true_cluster_label=true_cluster_label)
event_log.load(event_log_path + '/' + file_name, False)
event_log.preprocess()

## Representation Learning

In [15]:
# hyperparameters
n_epochs = 10
n_batch_size = 64
n_clusters = 5

vector_size = 32

### LSTM / GRU

In [16]:
# init and train LSTM
predictor = EmbeddingPredict(event_log)
predictor.build_model(embedding_dim=vector_size, gru_dim=vector_size, rnn='LSTM')
predictor.fit(epochs=n_epochs, batch_size=n_batch_size, verbose=True)

Epoch 1/10
8/8 [==============================] - 2s 4ms/step - loss: 24.9408 - dense_10_loss: 2.4646 - dense_11_loss: 2.7809 - dense_12_loss: 2.4516 - dense_13_loss: 2.2980 - dense_14_loss: 2.8275 - dense_15_loss: 2.3703 - dense_16_loss: 2.5558 - dense_17_loss: 2.3464 - dense_18_loss: 2.4883 - dense_19_loss: 2.3574 - dense_10_accuracy: 0.1420 - dense_11_accuracy: 0.1320 - dense_12_accuracy: 0.0980 - dense_13_accuracy: 0.2240 - dense_14_accuracy: 0.1500 - dense_15_accuracy: 0.1760 - dense_16_accuracy: 0.1080 - dense_17_accuracy: 0.2260 - dense_18_accuracy: 0.1580 - dense_19_accuracy: 0.3240 
Epoch 2/10
8/8 [==============================] - 0s 4ms/step - loss: 20.9032 - dense_10_loss: 2.1925 - dense_11_loss: 2.3128 - dense_12_loss: 2.0816 - dense_13_loss: 1.8024 - dense_14_loss: 2.3196 - dense_15_loss: 2.0354 - dense_16_loss: 2.0985 - dense_17_loss: 2.0066 - dense_18_loss: 2.1201 - dense_19_loss: 1.9336 - dense_10_accuracy: 0.1580 - dense_11_accuracy: 0.2080 - dense_12_accuracy: 0.1920

In [6]:
# get feature vector
pred_model, feature_vector, embedding_vector = predictor.predict()

### Clustering

In [7]:
# cluster feature vector
cluster_analysis = Clustering(event_log)
cluster_analysis.cluster(feature_vector, 'agglomerative', n_clusters, 'cosine')

cluster_result = cluster_analysis.evaluate()

In [8]:
print('Adjusted Rand Index: ' + str(cluster_result[0]))
print('Normalized Mutual Information: ' + str(cluster_result[1]))
print('F1-BCubed: ' + str(cluster_result[2]))

Adjusted Rand Index: 0.9908098820521286
Normalized Mutual Information: 0.9813041914061419
F1-BCubed: 0.9920901626047526


In [9]:
evaluation_a = Evaluation(event_log)
evaluation_a.evaluate_clusters(n_clusters,cluster_analysis._pred_labels)

C:\Users\malte\.conda\envs\replearn\lib\site-packages\pm4py\evaluation\replay_fitness\evaluator.py:73: UserWarning: Use the pm4py.algo.evaluation.replay_fitness package
  warnings.warn("Use the pm4py.algo.evaluation.replay_fitness package")
C:\Users\malte\.conda\envs\replearn\lib\site-packages\pm4py\evaluation\precision\evaluator.py:62: UserWarning: Use the pm4py.algo.evaluation.precision package
  warnings.warn("Use the pm4py.algo.evaluation.precision package")
C:\Users\malte\.conda\envs\replearn\lib\site-packages\pm4py\evaluation\simplicity\evaluator.py:38: UserWarning: Use the pm4py.algo.evaluation.simplicity package
  warnings.warn("Use the pm4py.algo.evaluation.simplicity package")


replaying log with TBR, completed variants ::   0%|          | 0/292 [00:00<?, ?it/s]

C:\Users\malte\.conda\envs\replearn\lib\site-packages\pm4py\evaluation\precision\evaluator.py:62: UserWarning: Use the pm4py.algo.evaluation.precision package
  warnings.warn("Use the pm4py.algo.evaluation.precision package")


replaying log with TBR, completed variants ::   0%|          | 0/1245 [00:00<?, ?it/s]

C:\Users\malte\.conda\envs\replearn\lib\site-packages\pm4py\evaluation\simplicity\evaluator.py:38: UserWarning: Use the pm4py.algo.evaluation.simplicity package
  warnings.warn("Use the pm4py.algo.evaluation.simplicity package")
C:\Users\malte\.conda\envs\replearn\lib\site-packages\pm4py\evaluation\replay_fitness\evaluator.py:73: UserWarning: Use the pm4py.algo.evaluation.replay_fitness package
  warnings.warn("Use the pm4py.algo.evaluation.replay_fitness package")


...

replaying log with TBR, completed variants ::   0%|          | 0/12 [00:00<?, ?it/s]

replaying log with TBR, completed variants ::   0%|          | 0/63 [00:00<?, ?it/s]

replaying log with TBR, completed variants ::   0%|          | 0/126 [00:00<?, ?it/s]

replaying log with TBR, completed variants ::   0%|          | 0/526 [00:00<?, ?it/s]

(0.8248620900710927, 0.8022950730482481, 0.5464288440736238)

In [19]:
import gzip
import json

with gzip.open('../logs/multi-perspective_02/huge_500_10_20_5_2_1-0.1-1.json.gz', "r") as f:
   data = f.read()
   j = json.loads (data.decode('utf-8'))
   print(json.dumps(j, indent=4, sort_keys=True))


{
    "attributes": {
        "generation_parameters": {
            "activity_dependency_p": 0.25,
            "anomalies": [
                {
                    "anomaly": "SkipSequence",
                    "parameters": {
                        "activities": [
                            "Activity A",
                            "Activity AA",
                            "Activity AB",
                            "Activity AC",
                            "Activity AD",
                            "Activity AE",
                            "Activity AF",
                            "Activity AG",
                            "Activity AH",
                            "Activity AI",
                            "Activity AJ",
                            "Activity AK",
                            "Activity AL",
                            "Activity AM",
                            "Activity AN",
                            "Activity AO",
                            "Activity AP",
  